### Fase 5: Creación de una Base de Datos
Organiza toda la información recopilada en una base de datos SQL bien estructurada. Define las tablas y relaciones necesarias para almacenar los datos de manera eficiente.


### Fase 6: Inserción de Datos en la Base de Datos
Inserta todos los datos recopilados en la base de datos diseñada.

___

In [3]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import funciones_auxiliares as fa

In [4]:
conexion = fa.conexion_bbdd("peliculas_imbd")
cursor = conexion.cursor()

In [ ]:
fa.creacion_tablas(conexion, cursor)